In [16]:
import pandas as pd
import numpy as np
import time
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report
import pandas as pd
import numpy as np
from sklearn.neural_network import MLPClassifier

In [ ]:
class Report:

    def __init__(self):
        self.report_folds = []

    def register_report_fold(self, y, y_pred):
        report = classification_report(y, y_pred, labels=None, target_names=None, sample_weight=None, digits=2, output_dict=True, zero_division='warn')
        report_fold = [report['accuracy'], report['1.0']['recall'], report['0.0']['recall']]
        self.report_folds.append(report_fold)

In [20]:
base = np.load(f"output/bins/bins_30.npy")

afib = base[base[:, -1] == 1]

snr = base[base[:, -1] == -1]
snr[:, -1] = np.zeros((snr.shape[0],))

base_ready = np.vstack((afib, snr))

y = base_ready[:, -1]
X = base_ready[:, 0:-1]

model = MLPClassifier(
              activation='tanh', 
              alpha=2.615276508013251e-06, 
              beta_1=0.999,
              beta_2=0.9, 
              hidden_layer_sizes=(30, 30),
              learning_rate_init=0.0009490229579734199, 
              max_iter=32,
              n_iter_no_change=32, 
              random_state=1, 
              validation_fraction=0.0,
              verbose=0, warm_start=True)

skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=1)


report_all = []
y_test_all = np.array([])
y_pred_all = np.array([])

start = time.time()
for train_index, test_index in skf.split(X, y):
    x_train_fold, x_test_fold = X[train_index], X[test_index]
    y_train_fold, y_test_fold = y[train_index], y[test_index]
    model.fit(x_train_fold, y_train_fold)
    y_pred = model.predict(x_test_fold)
    y_test_all = np.concatenate((y_test_all, y_test_fold))
    y_pred_all = np.concatenate((y_pred_all, y_pred))

    report = classification_report(y_test_fold, y_pred, labels=None, target_names=None, sample_weight=None, digits=2, output_dict=True, zero_division='warn')
    label_acc = report['accuracy']
    label_sen = report['1.0']['recall']
    label_spe = report['0.0']['recall']
    report_fold = [label_acc, label_sen, label_spe]
    print(report_fold)
    report_all.append(report_fold)

end = time.time()
print('Execution time:', time.strftime("%H:%M:%S", time.gmtime(end - start)))

report = classification_report(y_test_all, y_pred_all, labels=None, target_names=None, sample_weight=None, digits=2, output_dict=True, zero_division='warn')
label_acc_resume = f"{report['accuracy']}+-{np.array([r[0] for r in report_all]).std().round(4)}"
label_sen_resume = f"{report['1.0']['recall']}+-{np.array([r[1] for r in report_all]).std().round(4)}"
label_spe_resume = f"{report['0.0']['recall']}+-{np.array([r[2] for r in report_all]).std().round(4)}"
report_fold = [label_acc_resume, label_sen_resume, label_spe_resume]
report_all.append(report_fold)

indexes = list(range(1, 11))
indexes.append('Average')
pd.DataFrame(report_all, columns=['Accuracy', 'Sensibility', 'Specificity'], index=indexes)

/home/renan/Área de Trabalho/research-project-2/venv/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (32) reached and the optimization hasn't converged yet.
  warnings.warn(


[0.9726512760892432, 0.9770783157545054, 0.9637994021919628]
[0.9735924265072247, 0.9788223569470974, 0.9631351710395217]
[0.9734263411393457, 0.979569803172494, 0.9611424775821986]
[0.9750871948181365, 0.9789036544850498, 0.9674580773700814]
[0.9746443005037922, 0.9818106312292358, 0.9603187780175992]
[0.974035320821569, 0.9799833887043189, 0.9621451104100947]
[0.9751425566074295, 0.9800664451827242, 0.9652996845425867]
[0.9719869346177269, 0.9782392026578073, 0.9594886269301013]
[0.9719853836784409, 0.9790697674418605, 0.9578213218199934]
[0.9759162883401616, 0.9813953488372092, 0.9649618067087347]
Execution time: 00:02:29


,Accuracy,Sensibility,Specificity
1,0.972651,0.977078,0.963799
2,0.973592,0.978822,0.963135
3,0.973426,0.97957,0.961142
4,0.975087,0.978904,0.967458
5,0.974644,0.981811,0.960319
6,0.974035,0.979983,0.962145
7,0.975143,0.980066,0.9653
8,0.971987,0.978239,0.959489
9,0.971985,0.97907,0.957821
10,0.975916,0.981395,0.964962


: 

In [15]:
pd.crosstab(y_test_all, y_pred_all, rownames=['real'], colnames=['predict'], margins=True)

predict,0.0,1.0,All
real,,,
0.0,25370,441,25811
1.0,603,138576,139179
All,25973,139017,164990
